## Simple MCP

In [ ]:
from fastmcp import FastMCP

mcp = FastMCP(name="ToolServer")

@mcp.tool()
def get_full_name(first_name: str, last_name: str) -> int:
    """This function returns the full name of the person, given the first name and last name"""
    middle_name = "manoj"
    full_name = first_name + " " + middle_name + " " + last_name
    return full_name

if __name__ == "__main__":
    mcp.run()

In [ ]:
from fastmcp import Client
import asyncio

async def main():
    # Connect via stdio to a local script
    async with Client("mcp_server.py") as client:
        tools = await client.list_tools()
        print(f"Available tools: {tools}")
        result = await client.call_tool("get_full_name", {"first_name":"sagar", "last_name": "patil"})
        print(f"Result: {result}")

    # Connect via SSE
    # async with Client("http://localhost:8000/sse") as client:
    #     # ... use the client
    #     pass

if __name__ == "__main__":
    asyncio.run(main())

## multi tool single mcp

In [ ]:
from fastmcp import FastMCP

mcp = FastMCP(name="MultiToolServer")

@mcp.tool()
def get_full_name(first_name: str, last_name: str) -> int:
    """This function returns the full name of the person, given the first name and last name"""
    middle_name = "manoj"
    full_name = first_name + " " + middle_name + " " + last_name
    return full_name

@mcp.tool()
def add_and_multiply(a: int, b: int, c: int) -> int:
    """add and multiply function"""

    res = a + b * c

    return res

if __name__ == "__main__":
    mcp.run()

In [ ]:
from fastmcp import Client
import asyncio

async def main():
    # Connect via stdio to a local script
    async with Client("mcp_server.py") as client:
        tools = await client.list_tools()
        print(f"Available tools: {tools}")
        result = await client.call_tool("get_full_name", {"first_name":"sagar", "last_name": "patil"})
        print(f"\n\nResult: {result}")

        result = await client.call_tool("add_and_multiply", {"a":2,"b":3,"c":4 })
        print(f"Result: {result}")

    # Connect via SSE
    # async with Client("http://localhost:8000/sse") as client:
    #     # ... use the client
    #     pass

if __name__ == "__main__":
    asyncio.run(main())

## MCP With stdio

In [ ]:
# File Name: mcp_server.py

from fastmcp import FastMCP

mcp = FastMCP(name="GetNameMCPServer")

@mcp.tool()
async def get_full_name(first_name: str, last_name: str) -> int:
    """This function returns the full name of the person, given the first name and last name"""
    middle_name = "manoj"
    full_name = first_name + " " + middle_name + " " + last_name
    return full_name


if __name__ == "__main__":
    mcp.run()

In [ ]:
# File Name: mcp_server1.py

from fastmcp import FastMCP

mcp = FastMCP(name="WeatherMCPServer")

@mcp.tool()
async def get_weather(lat: int, lon: int) -> str:
    """This function returns the full name of the person, given the first name and last name"""
    print(f"latitude: {lat} \t longitude: {lon}")
    return "10 degree celcius"

if __name__ == "__main__":
    mcp.run()

In [ ]:
# File Name: mcp_client.py

from fastmcp import Client
import asyncio

config = {
    "mcpServers": {
        "getname": {"command": "python", "args": ["./mcp_server.py"]},
        "getweather": {"command": "python", "args": ["./mcp_server1.py"]},

    }
}
# Create a client that connects to all servers
client = Client(config)


async def main():
    # Connect via stdio to a local script
    async with client:
        tools = await client.list_tools()
        print(f"Available tools: {tools}")
        
        res = await client.call_tool(name="getname_get_full_name", arguments={"first_name":"sagar", "last_name": "patil"})

        print("\n\n Get Full Name: \n")
        print(res)

        res = await client.call_tool(name="getweather_get_weather", arguments={"lat":100000, "lon": 200000})

        print("\n\n Get Weather: \n")
        print(res)

if __name__ == "__main__":
    asyncio.run(main())

## MCP as a server

In [ ]:
# File Name: mcp_server1.py

from mcp.server.fastmcp import FastMCP
import asyncio

mcp = FastMCP(name="GetNameMCPServer", host="127.0.0.1", port=8001)

@mcp.tool()
async def get_full_name(first_name: str, last_name: str) -> int:
    """This function returns the full name of the person, given the first name and last name"""
    middle_name = "manoj"
    full_name = first_name + " " + middle_name + " " + last_name
    return full_name


# if __name__ == "__main__":
#     mcp.run(transport="sse")

async def main():
    await mcp.run_sse_async()

if __name__ == "__main__":
    asyncio.run(main())


In [ ]:
# File Name: mcp_server2.py

from mcp.server.fastmcp import FastMCP
import asyncio


mcp = FastMCP(name="WeatherMCPServer", host="127.0.0.1", port=8000)

@mcp.tool()
async def get_weather(lat: int, lon: int) -> str:
    """This function returns the full name of the person, given the first name and last name"""
    print(f"latitude: {lat} \t longitude: {lon}")
    return "10 degree celcius"

# if __name__ == "__main__":
#     mcp.run(transport="sse")

async def main():
    await mcp.run_sse_async()

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
# File Name: mcp_client.py

from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools
import asyncio

config = {
    "GetNameMCPServer": {
        "url": "http://127.0.0.1:8001/sse",
        "transport": "sse",
    },
    "WeatherMCPServer": {
        "url": "http://127.0.0.1:8000/sse",
        "transport": "sse",
    },

}

async def main():

    client = MultiServerMCPClient(config)
    tools = await client.get_tools()

    # print(tools)

    async with client.session("GetNameMCPServer") as session:
        tools1 = await load_mcp_tools(session)
        # print(tools1)
        res = await session.call_tool(name="get_full_name", arguments={"first_name":"sagar", "last_name": "patil"})

        print(res)


asyncio.run(main())

In [ ]:
# File Name: mcp_client.py

from fastmcp import Client
import asyncio

config = {
    "mcpServers": {
        "getname": {
            "url": "http://127.0.0.1:8001/sse",
            "transport": "sse",
        },
        "getweather": {
            "url": "http://127.0.0.1:8000/sse",
            "transport": "sse",
        },

    }
}

client_config = Client(config)

async def main():

    async with client_config:
        tools = await client_config.list_tools()
        print(tools)

        res = await client_config.call_tool(name="getname_get_full_name", arguments={"first_name":"sagar", "last_name": "patil"})

        print("\n\n")
        print(res)

        res = await client_config.call_tool(name="getweather_get_weather", arguments={"lat":100000, "lon": 200000})

        print("\n\n")
        print(res)

if __name__ == "__main__":
    asyncio.run(main())



## streamable-http MCP Serer

In [ ]:
# File Name: mcp_server1.py
from mcp.server.fastmcp import FastMCP
import asyncio

mcp = FastMCP(name="GetNameMCPServer", host="127.0.0.1", port=8001, stateless_http=True)

@mcp.tool()
async def get_full_name(first_name: str, last_name: str) -> int:
    """This function returns the full name of the person, given the first name and last name"""
    middle_name = "manoj"
    full_name = first_name + " " + middle_name + " " + last_name
    return full_name


# if __name__ == "__main__":
#     mcp.run(transport="sse")

async def main():
    await mcp.run_streamable_http_async()

if __name__ == "__main__":
    asyncio.run(main())


In [ ]:
# File Name: mcp_server2.py
from mcp.server.fastmcp import FastMCP
import asyncio


mcp = FastMCP(name="WeatherMCPServer", host="127.0.0.1", port=8000, stateless_http=True)

@mcp.tool()
async def get_weather(lat: int, lon: int) -> str:
    """This function returns the full name of the person, given the first name and last name"""
    print(f"latitude: {lat} \t longitude: {lon}")
    return "10 degree celcius"

# if __name__ == "__main__":
#     mcp.run(transport="sse")

async def main():
    await mcp.run_streamable_http_async()

if __name__ == "__main__":
    asyncio.run(main())


In [ ]:
# File Name: mcp_client.py

from fastmcp import Client
import asyncio

config = {
    "mcpServers": {
        "getname": {
            "url": "http://127.0.0.1:8001/mcp",
            "transport": "streamable-http",
        },
        "getweather": {
            "url": "http://127.0.0.1:8000/mcp",
            "transport": "streamable-http",
        },
        # "getname": {"command": "python", "args": ["./mcp_server.py"]},
        # "getweather": {"command": "python", "args": ["./mcp_server1.py"]},

    }
}
# Create a client that connects to all servers
client = Client(config)


async def main():
    # Connect via stdio to a local script
    async with client:
        tools = await client.list_tools()
        print(f"Available tools: {tools}")
        
        res = await client.call_tool(name="getname_get_full_name", arguments={"first_name":"sagar", "last_name": "patil"})

        print("\n\n Get Full Name: \n")
        print(res)

        res = await client.call_tool(name="getweather_get_weather", arguments={"lat":100000, "lon": 200000})

        print("\n\n Get Weather: \n")
        print(res)

if __name__ == "__main__":
    asyncio.run(main())


## MCP with llm as a tool call

In [ ]:
# File Name: mcp_server1.py
from mcp.server.fastmcp import FastMCP
import asyncio

mcp = FastMCP(name="GetNameMCPServer", host="127.0.0.1", port=8001, stateless_http=True)

@mcp.tool()
async def get_full_name(first_name: str, last_name: str) -> int:
    """This function returns the full name of the person, given the first name and last name"""
    middle_name = "manoj"
    full_name = first_name + " " + middle_name + " " + last_name
    return full_name


# if __name__ == "__main__":
#     mcp.run(transport="sse")

async def main():
    await mcp.run_streamable_http_async()

if __name__ == "__main__":
    asyncio.run(main())


In [ ]:
# File Name: mcp_server2.py
from mcp.server.fastmcp import FastMCP
import asyncio


mcp = FastMCP(name="WeatherMCPServer", host="127.0.0.1", port=8000, stateless_http=True)

@mcp.tool()
async def get_weather(lat: int, lon: int) -> str:
    """This function returns the full name of the person, given the first name and last name"""
    print(f"latitude: {lat} \t longitude: {lon}")
    return "10 degree celcius"

# if __name__ == "__main__":
#     mcp.run(transport="sse")

async def main():
    await mcp.run_streamable_http_async()

if __name__ == "__main__":
    asyncio.run(main())


In [ ]:
# File Name: client.py

import os
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_mcp_adapters.client import MultiServerMCPClient
import asyncio
from dotenv import load_dotenv
# Load .env values
load_dotenv()

config = {
        "getname": {
            "url": "http://127.0.0.1:8001/mcp",
            "transport": "streamable_http"
        },
        "getweather": {
            "url": "http://127.0.0.1:8000/mcp",
            "transport": "streamable_http"
        }

	}

async def main():
	llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=os.getenv("OPENAI_API_KEY"))

	mcp_client = MultiServerMCPClient(config)
	tools = await mcp_client.get_tools()

	agent = create_react_agent(llm, tools)

	
	# query = input("Query: ")
	query = "my name is sam and last name is altman give me my full name. also give me weather for latitude: 1000 and longitude: 2000 "
	result = await agent.ainvoke({"messages": [HumanMessage(content=query)]})
	print("Assistant:", result["messages"][-1].content)

if __name__ == "__main__":
   	asyncio.run(main())